# Mobile Price Classifications

In [ ]:
# Importing Useful Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
df_train = pd.read_csv('/kaggle/input/mobile-price-classification/train.csv')
df_test = pd.read_csv('/kaggle/input/mobile-price-classification/test.csv')
df_train.head(3).transpose()

In [ ]:
df_test.head()

In [ ]:
print('Train set shape ->', df_train.shape)
print('Test set shape ->', df_test.shape)

In [ ]:
df_train.describe().transpose()

In [ ]:
df_train.info()

In [ ]:
# Checking NaN value in the dataset
df_train.isnull().sum()

In [ ]:
# Let's Find out how many categories of Price Range
sns.countplot(x = 'price_range' , data = df_train)
plt.show()

In [ ]:
# Let's checck the Distribution of the data and Relationship between the Predictors

sns.pairplot(df_train , hue = 'price_range' , diag_kind= 'kde')
plt.show()

if you see the pair plot of the dataset along the Diagonally you will find the Distribution plot of the dataset You can see that only "Ram" Column Seperating Mobile price range to different classes, and every other data in ovelapping and also there is not normal distribution of data in dataset

Let's See the distribution of the data in the dataset

In [ ]:
sns.pairplot(df_train , diag_kind = 'kde')
plt.show()

## Let's the Hardware of the Mobile phones i.e "Battery_power", "clock_speed", "int_memory", "n_cores", "ram"

### ram

In [ ]:
sns.distplot(df_train['ram'])
plt.show()

In [ ]:
sns.histplot(df_train , x = 'ram' , hue = 'price_range' , kde = True)
plt.show()

In [ ]:
sns.histplot(df_train , x = 'ram' , hue = 'price_range' , multiple = 'dodge')
plt.show()

In [ ]:
sns.boxplot(x = 'price_range' , y = 'ram' , data = df_train)
plt.show()

In [ ]:
from statsmodels.stats.oneway import anova_oneway

result = anova_oneway(df_train['ram'] , groups= df_train['price_range'] , use_var='bf')
result

As we can see that the mean of column `ram` in the dataset in the `price_range` group Significate as the P-Value is `0` and from distribution of the Column we can conclude that the `ram` column devides data clearly in the Four Price range, So this column will be important in the classification process

## Internal Memory

In [ ]:
sns.displot( x = 'int_memory' , data = df_train , kind = 'kde')
plt.show()

In [ ]:
sns.histplot(df_train , x = 'int_memory' , hue = 'price_range' , multiple = 'dodge')
plt.show()

In [ ]:
sns.histplot(df_train , x = 'int_memory' , hue = 'price_range' , kde = True , multiple = 'dodge')
plt.show()

In [ ]:
sns.boxplot(data = df_train , x = 'price_range' , y = 'int_memory')
plt.show()

As we can see From distribution of the column `int_memory` that the Distribution overlap to each-other in and it cannot classify the data in the give price range

In [ ]:
# let's see the scatter plot of the int_memory and the ram
plt.figure(figsize=(12,8))
sns.scatterplot(data = df_train , x = 'ram' , y = 'int_memory' , hue = 'price_range')
plt.show()

From the scatter plot we can say that there is not Relationship between the internal memory and ram

In [ ]:
from mpl_toolkits import mplot3d

fig = plt.figure(figsize=(15 , 10))
ax = plt.axes(projection = '3d')

ax.scatter(df_train['battery_power'] , df_train['ram'] , df_train['int_memory'] , c = df_train['price_range'])
ax.set_xlabel('Battery Power')
ax.set_ylabel('Ram')
ax.set_zlabel('Internal Memory')
ax.legend()
plt.show()

## Let's see the "FC" column

The `fc` Columns have distribution skewed to left, so first let's find out the distribution

In [ ]:
sns.distplot(df_train['fc'])
plt.show()

In [ ]:
sns.boxplot(x = 'price_range' , y = 'fc' , data = df_train)
plt.show()

Let's see how many classes with in the data itself Using `Kmeans`

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist



df = df_train.drop('price_range',axis = 1)

distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
K = range(1, 10)
 
for k in K:
    # Building and fitting the model
    kmeanModel = KMeans(n_clusters=k).fit(df)
    kmeanModel.fit(df)
 
    distortions.append(sum(np.min(cdist(df, kmeanModel.cluster_centers_,
                                        'euclidean'), axis=1)) / df.shape[0])
    inertias.append(kmeanModel.inertia_)
 
    mapping1[k] = sum(np.min(cdist(df, kmeanModel.cluster_centers_,
                                   'euclidean'), axis=1)) / df.shape[0]
    mapping2[k] = kmeanModel.inertia_

In [ ]:
for key, val in mapping1.items():
    print(f'{key} : {val}')

In [ ]:
plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('The Elbow Method using Distortion')
plt.show()

As we can see that By `KMeans` The Elbow curve Sharply Decreases when value of K=2, That means data can be culster into 2 Group, i.e, It represents 2 Categories, But we have to Divide data into 4 categories

In [ ]:
# Let's find out the Correlation between columns

cor = df_train.corr()

In [ ]:
cor

In [ ]:
cor.style.background_gradient(cmap = 'coolwarm').set_precision(2)

**From the Correlation Co-efficient we can conclude following points.**
1. `PC` (Prime Camera Mega pixel) and `FC` (Front camera Mega pixel are strongly corelated, we have to see how this is affecting `Price_range`
2. `three_g` and `four_g` are also correlated that means when a Mobile has `3G` there is a probability that it also have `4G`, Now we have to find out how  having `3G` and `4G` connectivity together in the handset affect the price range
3. `sc_h` (screen height) and `sc_w` (screen width) also have a strong correlation, that means when height increase width also increse. Also `px_height` (pixel height) and `px_width` (pixel width) have strong correlation and the value of correlation coefficient is same as in `sc_h` and `sc_w` so there might be possibility that four column might be correlate to each other that we will find out with the help of `VIF`


**Let's First see the `pc` and `fc` Columns**

In [ ]:
sns.lmplot(x = 'pc' , y = 'fc' , data = df_train , hue = 'price_range')
plt.show()

In [ ]:
plt.figure(figsize = (12,8))

plt.subplot(121)
sns.histplot(data = df_train, x = 'pc' , hue = 'price_range' , kde = True , multiple = 'dodge')
plt.subplot(122)
sns.histplot(data = df_train , x = 'fc' , hue = 'price_range' , kde =True , multiple = 'dodge')
plt.tight_layout()
plt.show()

In [ ]:
plt.subplot(121)
sns.boxplot(x = 'price_range' , y = 'fc' , data = df_train)
plt.subplot(122)
sns.boxplot(x = 'price_range' , y = 'pc' , data = df_train)
plt.tight_layout()
plt.show()

Let's add both the Column and create a seperate column called `total_mp`

In [ ]:
df_train ['total_mp'] = df_train['pc'] + df_train['fc']
df_train.head(1)

In [ ]:
# Now lets see the distribution

sns.distplot(df_train['total_mp'])
plt.show()

In [ ]:
sns.boxplot(x = 'price_range' , y = 'total_mp' ,  data = df_train)
plt.show()

In [ ]:
sns.lmplot(x = 'total_mp' , y = 'fc' , data = df_train , hue = 'price_range')
plt.show()

In [ ]:
sns.lmplot(x = 'total_mp' , y = 'pc' , data = df_train , hue = 'price_range')
plt.show()

In [ ]:
sns.histplot(data = df_train , x = 'total_mp' , hue = 'price_range'  , kde = True , multiple = 'dodge')
plt.show()

**Let's see the `four_g` and `three_g` Column**

In [ ]:
sns.countplot(df_train['three_g'])
plt.show()

In [ ]:
sns.countplot(df_train['four_g'])
plt.show()

In [ ]:
sns.lmplot(x = 'three_g' , y = 'price_range' , data = df_train)
plt.show()

In [ ]:
sns.lmplot(x = 'four_g' , y = 'price_range' , data = df_train)
plt.show()

In [ ]:
# Let's Combine the Two columns

#df_train['sim_setting'] = df_train['three_g'].cat(df_train['four_g'] , sep = '_')
df_train['sim_setting'] = df_train[['three_g' , 'four_g']].astype('str').apply(lambda x : '_'.join(x) , axis = 1)
df_train.head()

In [ ]:
sns.countplot(df_train['sim_setting'])
plt.show()

In [ ]:
pd.pivot_table(df_train , index = ['sim_setting'] , columns = ['price_range'] , aggfunc= 'count').T

In [ ]:
sns.countplot(x = 'sim_setting' , hue = 'price_range' , data = df_train)
plt.show()

As you can see, the sim Configuaration do not have a much more affect on the price range. Though if `Mobile` has No `3G` or `4G` it's more likely to be in `0` Price range, If a Mobile have Only `3G` Sim confi. it more likely to come under the `2` Price range and If a Mobile Has Both `3G` & `4G` confi. It more likly to come Under `3` Price range then Others, Though the Number is not that significant.

**Let's analyze the columns `sc_w` , `sc_h` , `px_width` , `px_height`**

After seeing the correlation between `sc_w` & `sc_h` and `px_width` & `px_height` it seems that the `sc_w` and `px_width` and `sc_h` & `px_height` varies together.
So, test some whether the these are true or not

In [ ]:
# First let's test the "Mean" of the "sc_h"  & "px_height" whether Mean is same or different
# To do that we be using t-Test Hypothesis,
# The Null Hypothesis is that "the Mean of both Columns are Same"
# The Alternate Hypothesis will be that " they are not the same"
from  scipy.stats import ttest_rel

stats , p_value = ttest_rel(df_train['sc_h'] , df_train['px_height'])

print('p-value of the Paired t-Test of sc_h  , px_height ->>' , p_value)

In [ ]:
stats ,  P_value = ttest_rel(df_train['sc_w'] , df_train['px_width'])
print('p-value of the Paired t-Test of sc_h  , px_height ->>' , p_value)

As we can see that the P-Value is of the both Paired test is `0.0` which is less than significance level, Hence we reject the Null Hypothesis that the "Mean" values are not same

Let's check the "Mean" values of these four columns whether their Mean together have some Significance

In [ ]:
# we will be using "One way ANOVA" hypothesis
from statsmodels.stats.oneway import anova_oneway

result  = anova_oneway( df_train['px_width'], groups = df_train['price_range'], use_var= 'bf')

In [ ]:
result

In [ ]:
sns.histplot(data = df_train , x = 'px_width' ,hue =  'price_range' , multiple = 'dodge')
plt.show()

In [ ]:
sns.histplot(data = df_train, x = 'px_width' , hue = 'price_range' , multiple = 'dodge' , kde = True)
plt.show()

In [ ]:
result  = anova_oneway( df_train['px_height'], groups = df_train['price_range'], use_var= 'bf')
result

In [ ]:
plt.figure(figsize=(10,8))
sns.histplot(df_train , x = 'px_height' , hue = 'price_range' , multiple= 'dodge')
plt.show()

In [ ]:
sns.histplot(data = df_train , x = 'px_height' , hue = 'price_range' , kde = True , multiple = 'dodge')
plt.show()

In [ ]:
df_train['resolution'] = df_train['px_width'] + df_train['px_height']
df_train.head()

In [ ]:
result = anova_oneway(df_train['resolution'] , groups= df_train['price_range'] , use_var= 'bf')
result

In [ ]:
plt.figure(figsize = (10, 8))
sns.histplot(data = df_train ,  x = 'resolution' , hue = 'price_range' , kde = True , multiple = 'dodge')
plt.show()

# Let's See the physical features of the mobile i.e "sc_h" , "sc_w" , "m_dep" , "mobile_wt"

In [ ]:
# Mobile Height
anova_oneway(df_train['sc_h'] , groups = df_train['price_range'] , use_var= 'bf')

As the Mean of the Column `sc_h` in the group of `price_range` is not significant as the P-value is 8% which is greater than significance level

In [ ]:
# Mobile weight
anova_oneway(df_train['sc_w'] , groups = df_train['price_range'] , use_var= 'bf')

In [ ]:
# Mobile depth
anova_oneway(df_train['m_dep'] , groups = df_train['price_range'] , use_var = 'bf')

If we see the P-Value of the Columns `sc_h` , `sc_w` & `m_dep` in ANOVA Oneway Hypothesis test we found out that these 3 column's mean is not significantly different from the each other in `price_range` so, they cannot help in dividing the dataset in different price range

In [ ]:
# Mobile weight
anova_oneway(df_train['mobile_wt'] , groups = df_train['price_range'] , use_var = 'bf')

Now that P-Value for the mobile weight is 1% which is below the Significance Level and therefore the column `mobile_wt` can be an important column for the classification of mobile phones based on price 

In [ ]:
# Distribution of weight of the mobile
plt.figure(figsize = (12,8))
sns.histplot(data = df_train , x = 'mobile_wt' , hue = 'price_range' , kde = True , multiple = 'dodge')
plt.show()

The distribution of the Mobile weight columns ovarlaps each other in the groups

In [ ]:
# Let's Find out the volume of the mobile 

df_train['volume'] = df_train['sc_h'] * df_train['sc_w'] * df_train['mobile_wt']
df_train.head()

In [ ]:
# Let's see the mean and p-value of the "Volume" column

anova_oneway(df_train['volume'] , groups = df_train['price_range'] , use_var= 'bf')

In [ ]:
# Let's find out the volume/weight of the Mobile
df_train['density'] = round(df_train['volume'] / df_train['mobile_wt'])
df_train.head()

In [ ]:
# Now let's see the Mean and P-value of the density Column
anova_oneway(df_train['density'] , groups = df_train['price_range'] , use_var='bf')

In [ ]:
plt.figure(figsize=(12,8))
sns.histplot(data = df_train , x = 'density' , hue = 'price_range' , kde = True , multiple = 'dodge') 

Let's also find out that whether Mobile's Weight and Battery power are related or not 

In [ ]:
plt.figure(figsize=(12 , 8))
sns.scatterplot(data = df_train , x = 'mobile_wt' , y = 'battery_power' , hue = 'price_range')
plt.show()

There is no relationship between Mobile weight and Battery Power

In [ ]:
df_train.head().T
